In [1]:
from line_solver import *
GlobalConstants.set_verbose(VerboseLevel.SILENT)

# Clear variables (equivalent to MATLAB's clear P T E A solver AvgTable)
P = None
T = None 
E = None
A = None
solver = None
AvgTable = None

In [2]:
print('This example illustrates the execution on a layered queueing network model.')
print('Performance indexes now refer to processors, tasks, entries, and activities.')
print('Indexes refer to the submodel (layer) where the processor or task acts as a server.')
print('NaN indexes indicate that the metric is not supported by the node type.')

# cwd = os.path.dirname(os.path.abspath(__file__))
# model = LayeredNetwork.parseXML(os.path.join(cwd, 'example_layeredModel_1.xml'))

# Create model following MATLAB exactly
model = LayeredNetwork('myLayeredModel')

# MATLAB: P{1} = Processor(model, 'P1', 1, SchedStrategy.PS);
# MATLAB: P{2} = Processor(model, 'P2', 1, SchedStrategy.PS);
P = {}
P[1] = Processor(model, 'P1', 1, SchedStrategy.PS)
P[2] = Processor(model, 'P2', 1, SchedStrategy.PS)

# MATLAB: T{1} = Task(model, 'T1', 10, SchedStrategy.REF).on(P{1}).setThinkTime(Exp.fitMean(100));
# MATLAB: T{2} = Task(model, 'T2', 1, SchedStrategy.FCFS).on(P{2}).setThinkTime(Immediate());
T = {}
T[1] = Task(model, 'T1', 10, SchedStrategy.REF).on(P[1]).setThinkTime(Exp.fitMean(100))
T[2] = Task(model, 'T2', 1, SchedStrategy.FCFS).on(P[2]).setThinkTime(Immediate())

# MATLAB: E{1} = Entry(model, 'E1').on(T{1});
# MATLAB: E{2} = Entry(model, 'E2').on(T{2});
E = {}
E[1] = Entry(model, 'E1').on(T[1])
E[2] = Entry(model, 'E2').on(T[2])

# MATLAB: A{1} = Activity(model, 'AS1', Exp.fitMean(1.6)).on(T{1}).boundTo(E{1});
# MATLAB: A{2} = Activity(model, 'AS2', Immediate()).on(T{1}).synchCall(E{2},1);
# MATLAB: A{3} = Activity(model, 'AS3', Exp.fitMean(5)).on(T{2}).boundTo(E{2});
# MATLAB: A{4} = Activity(model, 'AS4', Exp.fitMean(1)).on(T{2}).repliesTo(E{2});
A = {}
A[1] = Activity(model, 'AS1', Exp.fitMean(1.6)).on(T[1]).boundTo(E[1])
A[2] = Activity(model, 'AS2', Immediate()).on(T[1]).synchCall(E[2], 1)
A[3] = Activity(model, 'AS3', Exp.fitMean(5)).on(T[2]).boundTo(E[2])
A[4] = Activity(model, 'AS4', Exp.fitMean(1)).on(T[2]).repliesTo(E[2])

# MATLAB: T{1}.addPrecedence(ActivityPrecedence.Serial(A{1}, A{2}));
# MATLAB: T{2}.addPrecedence(ActivityPrecedence.Serial(A{3}, A{4}));
T[1].addPrecedence(ActivityPrecedence.Serial(A[1], A[2]))
T[2].addPrecedence(ActivityPrecedence.Serial(A[3], A[4]))

This example illustrates the execution on a layered queueing network model.
Performance indexes now refer to processors, tasks, entries, and activities.
Indexes refer to the submodel (layer) where the processor or task acts as a server.
NaN indexes indicate that the metric is not supported by the node type.


In [3]:
# MATLAB: options = SolverLQNS.defaultOptions;
# MATLAB: options.keep = true; % uncomment to keep the intermediate XML files generates while translating the model to LQNS
options = SolverLQNS.defaultOptions()
options.keep = True  # uncomment to keep the intermediate XML files generates while translating the model to LQNS

# MATLAB: solver{1} = SolverLQNS(model);
# MATLAB: AvgTable{1} = solver{1}.get_avg_table();
# MATLAB: AvgTable{1}
solver = {}
AvgTable = {}

if SolverLQNS.isAvailable():
    solver[1] = SolverLQNS(model)
    AvgTable[1] = solver[1].get_avg_table()
    print('AvgTable[1]:')
    print(AvgTable[1])
else:
    print('LQNS solver not available')

# MATLAB: CallAvgTable = {};
# MATLAB: [AvgTable{2}, CallAvgTable{2}] = solver{1}.getRawAvgTables();
# MATLAB: AvgTable{2}
# MATLAB: CallAvgTable{2}
CallAvgTable = {}
if SolverLQNS.isAvailable():
    try:
        AvgTable[2], CallAvgTable[2] = solver[1].getRawAvgTables()
        print('AvgTable[2]:')
        print(AvgTable[2])
        print('CallAvgTable[2]:')
        print(CallAvgTable[2])
    except:
        print('getRawAvgTables method not yet implemented')

# MATLAB: AvgTable{3} = SolverLN(model).get_avg_table
AvgTable[3] = SolverLN(model).get_avg_table()
print('AvgTable[3]:')
print(AvgTable[3])

  Node   NodeType    QLen        Util    RespT  ResidT  ArvR    Tput
0   P1  Processor     NaN  1.4206e-01      NaN     0.0   0.0     NaN
1   P2  Processor     NaN  5.3271e-01      NaN     0.0   0.0     NaN
2   T1    RefTask  1.1215  1.4206e-01      NaN     0.0   0.0  0.0888
3   T2       Task  0.5327  5.3271e-01      NaN     0.0   0.0  0.0888
4   E1      Entry  1.1215  1.4206e-01  12.6310     0.0   0.0  0.0888
5   E2      Entry  0.5327  5.3271e-01   6.0000     0.0   0.0  0.0888
6  AS1   Activity  0.1627  1.4206e-01   1.8327     0.0   0.0  0.0888
7  AS2   Activity  0.9587  8.8786e-10  10.7983     0.0   0.0  0.0888
8  AS3   Activity  0.4439  4.4393e-01   5.0000     0.0   0.0  0.0888
9  AS4   Activity  0.0888  8.8786e-02   1.0000     0.0   0.0  0.0888
AvgTable[1]:
  Node   NodeType    QLen        Util    RespT  ResidT  ArvR    Tput
0   P1  Processor     NaN  1.4206e-01      NaN     0.0   0.0     NaN
1   P2  Processor     NaN  5.3271e-01      NaN     0.0   0.0     NaN
2   T1    RefTask  1.